In [ ]:
a!pip install turicreate

      Successfully uninstalled tensorflow-2.3.0
  Found existing installation: prettytable 1.0.1
    Uninstalling prettytable-1.0.1:
      Successfully uninstalled prettytable-1.0.1


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, sys
import datetime
import turicreate as tc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/AdventureW

/content/drive/My Drive/AdventureW


In [ ]:
#parse_dates = date_cols
date_cols = ["BirthDate","OrderDate","DueDate","ShipDate","DateFirstPurchase"]

# usecols = ["CustomerKey","ProductKey","OrderQuantity","MaritalStatus","Gender",
#            "TotalChildren","NumberChildrenAtHome","NumberCarsOwned","YearlyIncome",
#            ]

customers = pd.read_csv("Customers(FIS).csv", parse_dates=date_cols) # index_col='CustomerKey'

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#Drop Misiing Values
customers.dropna(axis=1, inplace=True)

In [ ]:
customers.shape

(60398, 49)

In [ ]:
# Extract Year of Birth
customers['Birth_Year']  = customers['BirthDate'].dt.year

#Map EnglishEducation to Numbers
customers['EnglishEducation'] = customers['EnglishEducation'].map({'Partial High School': 1,
                                                          'High School': 2,
                                                          'Partial College': 3,
                                                          'Bachelors': 4,
                                                          'Graduate Degree': 5, })

#Map EnglishOccupation to Numbers
customers['EnglishOccupation'] = customers['EnglishOccupation'].map({'Manual': 1,
                                'Skilled Manual': 2,
                                'Clerical': 3,
                                'Management': 4,
                                'Professional': 5,
                                })

#Map CommuteDistance to Numbers
customers['CommuteDistance'] = customers['CommuteDistance'].map({'0-1 Miles': 1,
                                                      '1-2 Miles': 2,
                                                      '2-5 Miles': 3,
                                                      '5-10 Miles': 4,
                                                      '10+ Miles': 5
                                                      })

In [ ]:
# OneHot Encoding
cat_feat = ['MaritalStatus','Gender'] # Remove SalesTerritoryKey
customers = pd.get_dummies(data=customers, columns=cat_feat, drop_first=True)

In [ ]:
columns_to_drop = ['GeographyKey','CustomerAlternateKey','FirstName','LastName',
                   'NameStyle','BirthDate','EmailAddress','SpanishEducation',
                   'FrenchEducation', 'SpanishOccupation', 'FrenchOccupation',
                   'AddressLine1','Phone','DateFirstPurchase','CurrencyKey','OrderDateKey',
                   'DueDateKey', 'ShipDateKey', 'CustomerKey.1','SalesOrderNumber', 'SalesOrderLineNumber',
                   'RevisionNumber','DueDate','OrderDate','ShipDate']

#Drop Certain Features
customers = customers.drop(columns_to_drop, axis=1)

In [ ]:
customers.head()

,CustomerKey,YearlyIncome,TotalChildren,NumberChildrenAtHome,EnglishEducation,EnglishOccupation,HouseOwnerFlag,NumberCarsOwned,CommuteDistance,ProductKey,PromotionKey,SalesTerritoryKey,OrderQuantity,UnitPrice,ExtendedAmount,UnitPriceDiscountPct,DiscountAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,Birth_Year,MaritalStatus_S,Gender_M
0,11015,30000.0,0,0,3,2,0,1,4,353,2,4,1,2319.99,2319.99,0,0,1265.6195,1265.6195,2319.99,185.5992,57.9998,1984,1,0
1,11015,30000.0,0,0,3,2,0,1,4,485,1,4,1,21.98,21.98,0,0,8.2205,8.2205,21.98,1.7584,0.5495,1984,1,0
2,11015,30000.0,0,0,3,2,0,1,4,486,1,4,1,159.00,159.00,0,0,59.4660,59.4660,159.00,12.7200,3.9750,1984,1,0
3,11017,20000.0,4,0,2,2,1,2,4,350,1,9,1,3374.99,3374.99,0,0,1898.0944,1898.0944,3374.99,269.9992,84.3748,1949,1,0
4,11017,20000.0,4,0,2,2,1,2,4,363,2,9,1,2294.99,2294.99,0,0,1251.9813,1251.9813,2294.99,183.5992,57.3748,1949,1,0


In [ ]:
customers_quant = customers.groupby(['CustomerKey','ProductKey']).sum()[['OrderQuantity']].reset_index()

In [ ]:
customers_quant[customers_quant['CustomerKey']==11001]

,CustomerKey,ProductKey,OrderQuantity
8,11001,217,1
9,11001,225,1
10,11001,350,1
11,11001,353,1
12,11001,477,2
13,11001,478,1
14,11001,479,1
15,11001,485,1
16,11001,491,1
17,11001,604,1


In [ ]:
print("Customers Shape - ", customers_quant.shape)

Customers Shape -  (59051, 3)


In [ ]:
customers_quant['Pro_dum'] = customers_quant['ProductKey']

In [ ]:
purchase_dummy = pd.get_dummies(customers_quant, columns=['Pro_dum'])

In [ ]:
purchase_dummy = purchase_dummy.melt(id_vars = purchase_dummy.columns[:3], var_name='ProductKey_dum', value_name='PurchaseDummy').drop('ProductKey_dum', axis=1)

In [ ]:
purchase_dummy

,CustomerKey,ProductKey,OrderQuantity,PurchaseDummy
0,11000,214,1,1
1,11000,344,1,0
2,11000,353,1,0
3,11000,485,1,0
4,11000,488,1,0
...,...,...,...,...
9330053,29480,479,1,0
9330054,29480,562,1,0
9330055,29481,349,1,0
9330056,29482,358,1,0


In [ ]:
customers_quant

,CustomerKey,ProductKey,OrderQuantity,Pro_dum
0,11000,214,1,214
1,11000,344,1,344
2,11000,353,1,353
3,11000,485,1,485
4,11000,488,1,488
...,...,...,...,...
59046,29480,479,1,479
59047,29480,562,1,562
59048,29481,349,1,349
59049,29482,358,1,358


In [ ]:
#Frequency Normalization

In [ ]:
customers_quant['Scaled_Frequency'] = customers_quant['OrderQuantity']/customers_quant['ProductKey'].nunique()

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(customers_quant[['Scaled_Frequency']])
scaled_df = pd.DataFrame(scaled_df)
customers_quant['Scaled_Frequency'] = scaled_df[0]

In [ ]:
customers_quant

,CustomerKey,ProductKey,OrderQuantity,Pro_dum,Scaled_Frequency
0,11000,214,1,214,-0.097741
1,11000,344,1,344,-0.097741
2,11000,353,1,353,-0.097741
3,11000,485,1,485,-0.097741
4,11000,488,1,488,-0.097741
...,...,...,...,...,...
59046,29480,479,1,479,-0.097741
59047,29480,562,1,562,-0.097741
59048,29481,349,1,349,-0.097741
59049,29482,358,1,358,-0.097741


In [ ]:
purchase_dummy

,CustomerKey,ProductKey,OrderQuantity,PurchaseDummy
0,11000,214,1,1
1,11000,344,1,0
2,11000,353,1,0
3,11000,485,1,0
4,11000,488,1,0
...,...,...,...,...
9330053,29480,479,1,0
9330054,29480,562,1,0
9330055,29481,349,1,0
9330056,29482,358,1,0


In [ ]:
#Convert Purchase count and Freqs Data to SFrame
customers_quant_sframe = customers_quant.to_dict(orient='records')
customers_quant_sframe = tc.SFrame(customers_quant_sframe)
customers_quant_sframe = customers_quant_sframe.unpack('X1', column_name_prefix='')

In [ ]:
#Convert Purchase Dummy Data to SFrame
purchase_dummy = purchase_dummy.to_dict(orient='records')
purchase_dummy = tc.SFrame(purchase_dummy)
purchase_dummy = purchase_dummy.unpack('X1', column_name_prefix='')

In [ ]:
# customers_quant_sframe.show()

# Cosine with Purchase Count

In [ ]:
#Split dataset for Count and Freqs
training_data, validation_data = tc.recommender.util.random_split_by_user(customers_quant_sframe, 'CustomerKey', 'ProductKey', item_test_proportion=0.2, random_seed=42)

#Build Model using Cosine Similarity
cosine_model = tc.item_similarity_recommender.create(training_data,
                                    user_id='CustomerKey',  
                                    item_id='ProductKey',
                                    target='OrderQuantity',
                                    similarity_type='cosine')

Warning: Ignoring columns Pro_dum, Scaled_Frequency;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 58400 observations with 18459 users and 158 items.

Data prepared in: 0.054853s

...

| 35.529ms                            | 100              | 158             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.05421s

In [ ]:
cosine_eval = cosine_model.evaluate(validation_data)


Precision and recall summary statistics by cutoff
+--------+---------------------+---------------------+
| cutoff |    mean_precision   |     mean_recall     |
+--------+---------------------+---------------------+
|   1    |  0.3068669527896995 | 0.22378397711015735 |
|   2    | 0.20815450643776823 |  0.2943133047210301 |
|   3    | 0.15951359084406286 |  0.3438483547925607 |
|   4    | 0.13948497854077246 |  0.3983905579399141 |
|   5    |  0.1218884120171674 |  0.4359442060085837 |
|   6    | 0.10872675250357657 |  0.4661659513590844 |
|   7    | 0.09993868792152055 |  0.4988912732474966 |
|   8    | 0.09066523605150213 |  0.5173104434907014 |
|   9    | 0.08273724368144969 |  0.5259656652360516 |
|   10   |  0.0757510729613734 |  0.5349785407725323 |
+--------+---------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 0.89552234754043

Per User RMSE (best)
+-------------+---------------------+-------+
| CustomerKey |         rmse        | count |
+---------

# Pearson with Purchase Count

In [ ]:
#Build Model using Pearson Similarity
pearson_model = tc.item_similarity_recommender.create(training_data,
                                    user_id='CustomerKey',  
                                    item_id='ProductKey',
                                    target='OrderQuantity',
                                    similarity_type='pearson')

Warning: Ignoring columns Pro_dum, Scaled_Frequency;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 58400 observations with 18459 users and 158 items.

Data prepared in: 0.069044s

...

| 51.462ms                            | 100              | 158             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.061613s

In [ ]:
pearson_eval = pearson_model.evaluate(validation_data)


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.051502145922746795 | 0.034155937052932746 |
|   2    | 0.045064377682403435 |  0.0621244635193133  |
|   3    | 0.04220314735336195  | 0.08608726752503576  |
|   4    | 0.04452789699570815  | 0.12525035765379108  |
|   5    | 0.043776824034334756 | 0.15475679542203152  |
|   6    | 0.049713876967095845 |  0.2082260371959943  |
|   7    | 0.049356223175965656 | 0.24370529327610863  |
|   8    | 0.04560085836909871  |  0.2551502145922746  |
|   9    | 0.04220314735336192  |   0.26552217453505   |
|   10   | 0.04098712446351933  | 0.28769670958512156  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.13715777113553612

Per User RMSE (best)
+-------------+------+-------+
| CustomerKey | rmse | count |
+--------

# Popularity with Purchase Count

In [ ]:

#Build a Popularity Model
popularity_model = tc.recommender.popularity_recommender.create(training_data,
                                                                user_id='CustomerKey',
                                                                item_id='ProductKey',
                                                                target='OrderQuantity',
                                                                random_seed=2020)

Warning: Ignoring columns Pro_dum, Scaled_Frequency;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 58400 observations with 18459 users and 158 items.

Data prepared in: 0.066618s

58400 observations to process; with 158 unique items.

In [ ]:
popularity_eval = popularity_model.evaluate(validation_data)


Precision and recall summary statistics by cutoff
+--------+----------------------+---------------------+
| cutoff |    mean_precision    |     mean_recall     |
+--------+----------------------+---------------------+
|   1    | 0.06437768240343346  | 0.04327610872675251 |
|   2    | 0.048283261802575125 | 0.06695278969957082 |
|   3    | 0.04363376251788269  | 0.08930615164520746 |
|   4    | 0.04345493562231761  |  0.1209585121602289 |
|   5    | 0.04892703862660942  | 0.16859799713876966 |
|   6    | 0.05901287553648067  | 0.24907010014306155 |
|   7    | 0.05702023298589823  |  0.2830472103004291 |
|   8    |  0.0523068669527897  | 0.29806866952789696 |
|   9    | 0.047925608011444874 |  0.3077253218884121 |
|   10   | 0.04849785407725318  | 0.34724606580829764 |
+--------+----------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 0.13923879794732041

Per User RMSE (best)
+-------------+------+-------+
| CustomerKey | rmse | count |
+-------------+------+-

# Cosine with Purchase Dummy

In [ ]:
purchase_dummy

CustomerKey,OrderQuantity,ProductKey,PurchaseDummy
11000,1,214,1
11000,1,344,0
11000,1,353,0
11000,1,485,0
11000,1,488,0
11000,1,530,0
11000,1,541,0
11000,1,573,0
11001,1,217,0
11001,1,225,0


In [ ]:
#Split dataset for dummy
training_data_dum, validation_data_dum = tc.recommender.util.random_split_by_user(purchase_dummy, 'CustomerKey', 'ProductKey', item_test_proportion=0.2, random_seed=42)
# user_data_dum = training_data_dum[['CustomerKey','OrderQuantity']]


In [ ]:
#Build Model using Cosine Similarity
cosine_model_dum = tc.item_similarity_recommender.create(training_data_dum,
                                    user_id='CustomerKey',  
                                    item_id='ProductKey',
                                    target='PurchaseDummy',
                                    # user_data=user_data_dum,
                                    similarity_type='cosine')

Warning: Ignoring columns OrderQuantity;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 9229452 observations with 18484 users and 158 items.

Data prepared in: 7.22239s

...

| 47.785ms                            | 100              | 158             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.429266s

In [ ]:
cosine_eval_dum = cosine_model_dum.evaluate(validation_data_dum)

recommendations finished on 1000/1000 queries. users per second: 67934.8


Precision and recall summary statistics by cutoff
+--------+----------------+-------------+
| cutoff | mean_precision | mean_recall |
+--------+----------------+-------------+
|   1    |      0.0       |     0.0     |
|   2    |      0.0       |     0.0     |
|   3    |      0.0       |     0.0     |
|   4    |      0.0       |     0.0     |
|   5    |      0.0       |     0.0     |
|   6    |      0.0       |     0.0     |
|   7    |      0.0       |     0.0     |
|   8    |      0.0       |     0.0     |
|   9    |      0.0       |     0.0     |
|   10   |      0.0       |     0.0     |
+--------+----------------+-------------+
[10 rows x 3 columns]


Overall RMSE: 0.07959602383417265

Per User RMSE (best)
+-------------+------+-------+
| CustomerKey | rmse | count |
+-------------+------+-------+
|    23838    | 0.0  |   40  |
+-------------+------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+-------------+---------------------+-------+
| CustomerKey |         rmse       

# Pearson with Purchase Dummy

In [ ]:
#Build Model using Pearson Similarity
pearson_model_dum = tc.item_similarity_recommender.create(training_data_dum,
                                    user_id='CustomerKey',  
                                    item_id='ProductKey',
                                    target='PurchaseDummy',
                                    # user_data=user_data_dum,
                                    similarity_type='pearson')

Warning: Ignoring columns OrderQuantity;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 9229452 observations with 18484 users and 158 items.

Data prepared in: 7.1576s

...

| 40.679ms                            | 100              | 158             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.421438s

In [ ]:
pearson_eval_dum = pearson_model_dum.evaluate(validation_data_dum)

recommendations finished on 1000/1000 queries. users per second: 56967.1


Precision and recall summary statistics by cutoff
+--------+----------------+-------------+
| cutoff | mean_precision | mean_recall |
+--------+----------------+-------------+
|   1    |      0.0       |     0.0     |
|   2    |      0.0       |     0.0     |
|   3    |      0.0       |     0.0     |
|   4    |      0.0       |     0.0     |
|   5    |      0.0       |     0.0     |
|   6    |      0.0       |     0.0     |
|   7    |      0.0       |     0.0     |
|   8    |      0.0       |     0.0     |
|   9    |      0.0       |     0.0     |
|   10   |      0.0       |     0.0     |
+--------+----------------+-------------+
[10 rows x 3 columns]


Overall RMSE: 0.079388753345016

Per User RMSE (best)
+-------------+----------------------+-------+
| CustomerKey |         rmse         | count |
+-------------+----------------------+-------+
|    28166    | 0.006237182517135713 |   32  |
+-------------+----------------------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+--

# Popularity with Purchase Dummy

In [ ]:
#Build a Popularity Model
popularity_model = tc.recommender.popularity_recommender.create(training_data_dum,
                                                                user_id='CustomerKey',
                                                                item_id='ProductKey',
                                                                target='PurchaseDummy',
                                                                # user_data=user_data_dum,
                                                                random_seed=2020)

Warning: Ignoring columns OrderQuantity;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 9229452 observations with 18484 users and 158 items.

Data prepared in: 8.48149s

9229452 observations to process; with 158 unique items.

In [ ]:
popularity_eval_dum = popularity_model.evaluate(validation_data_dum)

recommendations finished on 1000/1000 queries. users per second: 76493.5


Precision and recall summary statistics by cutoff
+--------+----------------+-------------+
| cutoff | mean_precision | mean_recall |
+--------+----------------+-------------+
|   1    |      0.0       |     0.0     |
|   2    |      0.0       |     0.0     |
|   3    |      0.0       |     0.0     |
|   4    |      0.0       |     0.0     |
|   5    |      0.0       |     0.0     |
|   6    |      0.0       |     0.0     |
|   7    |      0.0       |     0.0     |
|   8    |      0.0       |     0.0     |
|   9    |      0.0       |     0.0     |
|   10   |      0.0       |     0.0     |
+--------+----------------+-------------+
[10 rows x 3 columns]


Overall RMSE: 0.07938219124111892

Per User RMSE (best)
+-------------+----------------------+-------+
| CustomerKey |         rmse         | count |
+-------------+----------------------+-------+
|    28166    | 0.006256765533665294 |   32  |
+-------------+----------------------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+

# Cosine with Scaled Frequency

In [ ]:
#Split dataset for Count and Freqs
training_data_freq, validation_data_freq = tc.recommender.util.random_split_by_user(customers_quant_sframe, 'CustomerKey', 'ProductKey', item_test_proportion=0.2, random_seed=42)

#Build Model using Cosine Similarity
cosine_model_freq = tc.item_similarity_recommender.create(training_data_freq,
                                    user_id='CustomerKey',  
                                    item_id='ProductKey',
                                    target='Scaled_Frequency',
                                    similarity_type='cosine')

Warning: Ignoring columns OrderQuantity, Pro_dum;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 58400 observations with 18459 users and 158 items.

Data prepared in: 0.071732s

...

| 41.943ms                            | 100              | 158             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.057974s

In [ ]:
cosine_eval_freq = cosine_model_freq.evaluate(validation_data_freq)


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.010729613733905583 | 0.009298998569384837 |
|   2    | 0.007510729613733905 | 0.010801144492131614 |
|   3    | 0.006437768240343349 | 0.013662374821173103 |
|   4    | 0.006974248927038629 | 0.019027181688125894 |
|   5    | 0.006437768240343347 | 0.023319027181688136 |
|   6    | 0.006080114449213162 | 0.027610872675250357 |
|   7    | 0.006131207847946042 | 0.03233190271816881  |
|   8    | 0.006974248927038628 | 0.039306151645207434 |
|   9    | 0.006914639961850264 | 0.04359799713876965  |
|   10   | 0.006866952789699572 | 0.04788984263233191  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.5848713443290944

Per User RMSE (best)
+-------------+----------------------+-------+
| CustomerKey |         r

# Pearson with Scaled Frequency

In [ ]:
#Build Model using Pearson Similarity
pearson_model_freq = tc.item_similarity_recommender.create(training_data_freq,
                                    user_id='CustomerKey',  
                                    item_id='ProductKey',
                                    target='Scaled_Frequency',
                                    similarity_type='pearson')

Warning: Ignoring columns OrderQuantity, Pro_dum;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 58400 observations with 18459 users and 158 items.

Data prepared in: 0.071491s

...

| 33.428ms                            | 100              | 158             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.046156s

In [ ]:
pearson_eval_freq = pearson_model_freq.evaluate(validation_data_freq)


Precision and recall summary statistics by cutoff
+--------+----------------------+---------------------+
| cutoff |    mean_precision    |     mean_recall     |
+--------+----------------------+---------------------+
|   1    | 0.05793991416309014  | 0.03683834048640917 |
|   2    | 0.04721030042918455  | 0.06373390557939915 |
|   3    | 0.04363376251788269  | 0.08876967095851214 |
|   4    |  0.046137339055794   |  0.1290057224606581 |
|   5    | 0.04592274678111585  | 0.16173104434907007 |
|   6    | 0.053290414878397724 | 0.22449928469241764 |
|   7    | 0.05242182709993864  | 0.25997854077253196 |
|   8    | 0.048283261802575105 | 0.27249642346208863 |
|   9    | 0.04577968526466377  | 0.29145207439198845 |
|   10   | 0.04463519313304723  |  0.3179184549356223 |
+--------+----------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 0.5793090822117953

Per User RMSE (best)
+-------------+------+-------+
| CustomerKey | rmse | count |
+-------------+------+--

#Popularity with Scaled Frequency

In [ ]:
#Build a Popularity Model
popularity_model_freq = tc.recommender.popularity_recommender.create(training_data_freq,
                                                                user_id='CustomerKey',
                                                                item_id='ProductKey',
                                                                target='Scaled_Frequency',
                                                                random_seed=2020)

Warning: Ignoring columns OrderQuantity, Pro_dum;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 58400 observations with 18459 users and 158 items.

Data prepared in: 0.082221s

58400 observations to process; with 158 unique items.

In [ ]:
popularity_eval_freq = popularity_model_freq.evaluate(validation_data_freq)


Precision and recall summary statistics by cutoff
+--------+----------------------+---------------------+
| cutoff |    mean_precision    |     mean_recall     |
+--------+----------------------+---------------------+
|   1    | 0.06437768240343346  | 0.04327610872675249 |
|   2    | 0.048283261802575125 | 0.06695278969957082 |
|   3    | 0.043633762517882695 | 0.08930615164520746 |
|   4    | 0.04345493562231759  | 0.12095851216022892 |
|   5    | 0.048927038626609444 | 0.16859799713876972 |
|   6    | 0.05901287553648068  |  0.2490701001430615 |
|   7    | 0.05702023298589821  |  0.2830472103004291 |
|   8    |  0.0523068669527897  | 0.29806866952789707 |
|   9    | 0.04792560801144489  | 0.30772532188841195 |
|   10   | 0.04849785407725322  | 0.34724606580829764 |
+--------+----------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 0.5966194161659686

Per User RMSE (best)
+-------------+------+-------+
| CustomerKey | rmse | count |
+-------------+------+--

In [ ]:
#Performance of all models Using Precision and Recall 
tc.recommender.util.compare_models(validation_data, [cosine_model, pearson_model, popularity_model], metric='precision_recall')

In [ ]:
#Get recommended products for user
cosine_rec = cosine_model.recommend()

recommendations finished on 1000/18459 queries. users per second: 135153

recommendations finished on 2000/18459 queries. users per second: 137788

recommendations finished on 3000/18459 queries. users per second: 115942

recommendations finished on 4000/18459 queries. users per second: 115430

recommendations finished on 5000/18459 queries. users per second: 117922

...

recommendations finished on 14000/18459 queries. users per second: 125731

recommendations finished on 15000/18459 queries. users per second: 126607

recommendations finished on 16000/18459 queries. users per second: 127622

recommendations finished on 17000/18459 queries. users per second: 128330

recommendations finished on 18000/18459 queries. users per second: 128231

In [ ]:
cosine_rec.print_rows(num_rows = 40, num_columns = 4) 

+-------------+------------+---------------------+------+
| CustomerKey | ProductKey |        score        | rank |
+-------------+------------+---------------------+------+
|    11000    |    480     | 0.12689762562513351 |  1   |
|    11000    |    477     | 0.11075685918331146 |  2   |
|    11000    |    478     |  0.1043827012181282 |  3   |
|    11000    |    225     | 0.10306187719106674 |  4   |
|    11000    |    529     |  0.1012224331498146 |  5   |
|    11000    |    528     | 0.09604915231466293 |  6   |
|    11000    |    537     | 0.08558521419763565 |  7   |
|    11000    |    217     | 0.08470942825078964 |  8   |
|    11000    |    487     | 0.07671985030174255 |  9   |
|    11000    |    479     | 0.07189438492059708 |  10  |
|    11001    |    214     | 0.11822129487991333 |  1   |
|    11001    |    528     | 0.11363202929496766 |  2   |
|    11001    |    537     | 0.11137312650680542 |  3   |
|    11001    |    487     | 0.10360699892044067 |  4   |
|    11001    

In [ ]:
##Customer Recommendation function to excel

In [ ]:
#Convert Sframe of Recommended products to Pandas DataFrame
cosine_rec_df = cosine_rec.to_dataframe()

In [ ]:
#Save the Recommended Product dataframe to CSV
cosine_rec_df.to_csv('Final_Rec_Cosine_Sim_Count.csv', index=False)

In [ ]:
#Recommedn Product for a User
cosine_model.recommend(users=['11015'])

CustomerKey,ProductKey,score,rank
11015,478,0.12890752156575522,1
11015,480,0.10096220175425212,2
11015,528,0.09900734821955363,3
11015,537,0.09805210431416829,4
11015,477,0.09783838192621867,5
11015,214,0.07712111870447795,6
11015,217,0.07394182682037354,7
11015,222,0.07256937026977539,8
11015,225,0.07093872626622517,9
11015,529,0.06148775418599447,10


In [ ]:
#Recommedn Product for a User
popularity_model.recommend(users=['11015'])

CustomerKey,ProductKey,score,rank
11015,477,4195.0,1
11015,480,3159.0,2
11015,528,3048.0,3
11015,529,2341.0,4
11015,214,2202.0,5
11015,225,2165.0,6
11015,222,2098.0,7
11015,217,2066.0,8
11015,478,2000.0,9
11015,479,1692.0,10
